Importing data from Google Sheets

Referencing snippet from here: https://colab.research.google.com/notebooks/snippets/sheets.ipynb#scrollTo=JiJVCmu3dhFa 

In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [2]:
worksheet = gc.open('allRiotGlassDoor').sheet1

In [3]:
rows = worksheet.get_all_values()
print(rows)

[['title', 'author_info', 'rating', 'pros', 'cons', 'helpful'], ['One of the best companies out there!', 'Jul 10, 2018 - Character Artist', '5', 'Company which respects its employees and is pleasure to work at.', 'Cannot think of a con.', 'Be the first to find this review helpful'], ['Player Support', 'Jun 27, 2018 -', '3', 'Good company overall. Perks and the environment are good too', 'It has changed its procedures and it´s got too many policies and rules to follow. Which doesn´t help much with the criativity of the employers. What used to take a few days to get settled it takes too long loosing the purpose of the project most of the times.', 'Be the first to find this review helpful'], ['Your performance is solely based on relationships', 'Apr 27, 2018 -', '2', 'Very talented people work there. The product is awesome. The brand and reputation make you feel a rockstar. Salaries are good and benefits above market.', "If you don't build a feel-good relationship with specific senior man

Converting the spreadsheet to a Pandas Dataframe

In [4]:
import pandas as pd

In [5]:
riot_df = pd.DataFrame.from_records(rows)
display(riot_df)

,0,1,2,3,4,5
0,title,author_info,rating,pros,cons,helpful
1,One of the best companies out there!,"Jul 10, 2018 - Character Artist",5,Company which respects its employees and is pl...,Cannot think of a con.,Be the first to find this review helpful
2,Player Support,"Jun 27, 2018 -",3,Good company overall. Perks and the environmen...,It has changed its procedures and it´s got too...,Be the first to find this review helpful
3,Your performance is solely based on relationships,"Apr 27, 2018 -",2,Very talented people work there. The product i...,If you don't build a feel-good relationship wi...,10 people found this review helpful
4,GG Thanks for the Job,"Apr 22, 2018 - Anonymous Employee",5,Good location and a beautiful place,nothing so many important to say,1 person found this review helpful
...,...,...,...,...,...,...
585,There once was a dream that was Rome,"Apr 11, 2018 - Anonymous Employee",1,"In the interest of fairness, I want to make it...",The world changes and what made Riot great yea...,28 people found this review helpful
586,Great place for self starters,"Mar 2, 2018 - Talent in Los Angeles, CA",5,"- Opportunity for growth, I've worked at Riot ...","- There is so much work to do, that you could ...",5 people found this review helpful
587,For those looking to grow & challenge themselves,"Mar 6, 2018 - Program Manager in Santa Monica, CA",5,Riot puts a serious and genuine emphasis on bo...,"With the explosion of League of Legends, Riot'...",3 people found this review helpful
588,Riot can be what you make of it,"Mar 4, 2018 - Software Engineer in Los Angeles...",5,Pros: - Campus is a great environment - Lots o...,- Can be difficult if you aren't a self-starte...,2 people found this review helpful


In [6]:
# Designating the first row of the dataframe as the header
riot_df.columns = riot_df.iloc[0]
riot_df = riot_df[1:]
riot_df.head()

,title,author_info,rating,pros,cons,helpful
1,One of the best companies out there!,"Jul 10, 2018 - Character Artist",5,Company which respects its employees and is pl...,Cannot think of a con.,Be the first to find this review helpful
2,Player Support,"Jun 27, 2018 -",3,Good company overall. Perks and the environmen...,It has changed its procedures and it´s got too...,Be the first to find this review helpful
3,Your performance is solely based on relationships,"Apr 27, 2018 -",2,Very talented people work there. The product i...,If you don't build a feel-good relationship wi...,10 people found this review helpful
4,GG Thanks for the Job,"Apr 22, 2018 - Anonymous Employee",5,Good location and a beautiful place,nothing so many important to say,1 person found this review helpful
5,Amazing,"Apr 23, 2018 - Media & Community Lead in Ciuda...",5,- Amazing job culture - Focus on players - Cha...,"- Nothing, is the dream company",Be the first to find this review helpful


Cleaning up the dataframe by...

- Removing the `helpful` (last) column, which indicates how many Glassdoor users rated a review as "helpful." This information is not relevant to us.
- Parsing the date from the `author_info` (second) column. While job titles are not always provided – as review authors are not obligated by Glassdoor to state them — they are spliced out whenever they are. This allows us to only access the information we need: the date the review was posted.

In [7]:
# Remove last column by its title
riot_df = riot_df.drop('helpful', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [8]:
import datetime

In [9]:
# Helper function for date formatting (MM/DD/YY).
# i.e. Takes "Jan 1, 2000" as input, and returns "01/01/2000" as output.
# Note that both I/O are strings.
def format_date(original_date):

  date_components = original_date.split(' ')

  # Convert the month from abbreviated to numerical format.
  # Pad zeroes wherever appropriate.
  month_published = str(datetime.datetime.strptime(date_components[0], "%b").month).zfill(2)

  # Remove the trailing comma from the day (second item in list).
  # Again, pad zeroes wherever appropriate.
  day_published = date_components[1][0:-1].zfill(2)

  year_published = date_components[2]

  date_formatted = month_published + '/' + day_published + '/' + year_published
  return date_formatted

In [10]:
# Extract the date from the last column
for index, row in riot_df.iterrows():

  delimiter = ' - '
  split_info = row['author_info'].split(delimiter)
  
  date_published = ''

  # If a job title was provided by the reviewer, we splice it out.
  if len(split_info) > 1:
    date_published = split_info[0]

  # If no job title was provided, then the date is simply
  # the first item in the list, with the trailing space & hyphen excluded.
  # So exclude the last two characters.
  else:
    date_published = split_info[0][0:-2]
  
  # Format the date, relying on the helper function above.
  date_formatted = format_date(date_published)
  
  # Update the dataframe.
  riot_df.loc[index, 'author_info'] = date_formatted

In [11]:
riot_df.head()

,title,author_info,rating,pros,cons
1,One of the best companies out there!,07/10/2018,5,Company which respects its employees and is pl...,Cannot think of a con.
2,Player Support,06/27/2018,3,Good company overall. Perks and the environmen...,It has changed its procedures and it´s got too...
3,Your performance is solely based on relationships,04/27/2018,2,Very talented people work there. The product i...,If you don't build a feel-good relationship wi...
4,GG Thanks for the Job,04/22/2018,5,Good location and a beautiful place,nothing so many important to say
5,Amazing,04/23/2018,5,- Amazing job culture - Focus on players - Cha...,"- Nothing, is the dream company"


### Retroactive date handling (pt. 1)

In [12]:
import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from pprint import pprint

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [13]:
lemmatizer = WordNetLemmatizer()

def tokenizeLemmatize(reviews):
  temp = []
  for sentence in reviews:
    tokens = word_tokenize(sentence)
    cleanedSentence = ""
    for token in tokens:
        lemmetized_word = lemmatizer.lemmatize(token)
        cleanedSentence += lemmetized_word + " "
    temp.append(cleanedSentence)
  return temp

In [14]:
# Initializing a Python dictionary wherein
# keys: the tokenized, lemmatized review sentence
# values: the date (MM/DD/YY) that review was posted on

# Helper function
# Returns sentences as list
def parse_sentences_from_review_block(review):

  list_of_sentences = []
  
  current_sentence = ''
  previous_char = review[0]
  
  for character in review:
    
    # Encounter period -> assume sentence
    if character == '.':
      if current_sentence.strip('-. ') and not current_sentence.strip(',. ').isspace():
        list_of_sentences.append(current_sentence.strip('-.'))
      # Reset
      current_sentence = ''
    
    # Encounter hyphen -> assume sentence
    elif character == ' ' and previous_char == '-':
      if current_sentence.strip('-. ') and not current_sentence.strip(',. ').isspace():
        list_of_sentences.append(current_sentence.strip('.-'))
      # Reset
      current_sentence = ''
    
    # Continue
    current_sentence += character
    previous_char = character
  
  # Append whatever's left, if it hasn't already been caught
  if current_sentence.strip('-. ') and not current_sentence.strip(',. ').isspace():
    list_of_sentences.append(current_sentence.strip('.-'))
  
  return list_of_sentences

# column_name is either 'pros' or 'cons'
def create_dates_dictionary(df, column_name):
  dates_dict = {}

  for index, row in df.iterrows():

    date_string = row['author_info']
    review = row[column_name]

    review_sentences = parse_sentences_from_review_block(review)
    cleaned_sentences = tokenizeLemmatize(review_sentences)
    # print(cleaned_sentences)

    for each_sentence in cleaned_sentences:
      dates_dict[each_sentence] = date_string
    
  return dates_dict

In [15]:
dates_dict_pros = create_dates_dictionary(riot_df, "pros")
print(dates_dict_pros)

{'Company which respect it employee and is pleasure to work at ': '07/10/2018', 'Good company overall ': '06/27/2018', 'Perks and the environment are good too ': '06/27/2018', 'Very talented people work there ': '04/27/2018', 'The product is awesome ': '04/27/2018', 'The brand and reputation make you feel a rockstar ': '04/27/2018', 'Salaries are good and benefit above market ': '04/27/2018', 'Good location and a beautiful place ': '04/22/2018', 'Amazing job culture ': '04/23/2018', 'Focus on player ': '04/23/2018', 'Challenge convention ': '04/23/2018', 'Company Perks ': '04/10/2018', 'Talents ': '04/10/2018', 'Disruptive atmosphere ': '04/10/2018', 'Feedback culture ': '01/24/2018', 'Even already established since late 2013 , the Hong Kong Studio still breathes Start-Up ': '02/20/2018', 'It is small scale , we shifted away from being a China Publishing Office to become and build the first International Studio 2 year ago ': '02/20/2018', 'If you like to work in a Start-up environment 

In [16]:
dates_dict_cons = create_dates_dictionary(riot_df, "cons")
print(dates_dict_cons)

{'Can not think of a con ': '07/10/2018', 'It ha changed it procedure and it´s got too many policy and rule to follow ': '06/27/2018', 'Which doesn´t help much with the criativity of the employer ': '06/27/2018', 'What used to take a few day to get settled it take too long loosing the purpose of the project most of the time ': '06/27/2018', "If you do n't build a feel-good relationship with specific senior manager you will not thrive ": '04/27/2018', "Nice people that do n't do anything are better evaluated than hard working people rough around the edge ": '04/27/2018', 'Meritocracy is a myth ': '04/27/2018', 'nothing so many important to say ': '04/22/2018', 'Nothing , is the dream company ': '04/23/2018', 'No diversity ': '04/10/2018', 'Lack of management for people who work during weekend ': '04/10/2018', 'Lack of work-life balance ': '07/31/2013', 'We are still partly dependent on the Central product team at the moment wrt the product we are working on in Hong Kong ': '02/20/2018',

# Zero-shot classification

Now, we're going to use zero-shot classification to classify our reviews
according to these axes: Culture and Values, Diversity and Inclusion, Work/Life Balance, Senior Management, Compensation and Benefits, and Career Opportunities. 

To do this, we'll first make a long list of all the sentences from our reviews. 

Then, we'll use BART from Hugging Face (https://huggingface.co/facebook/bart-large-mnli) to classify those sentences, putting them into appropriate lists! We will also keep them separated by negative and positive by assuming that whatever is under "pros" is positive, and whatever is under "cons" can be expected to be negative--this will be useful later on when we begin to use BERT for sentiment analysis.

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.6 MB/s 
     |████████████████████████████████| 596 kB 36.1 MB/s 
     |████████████████████████████████| 6.6 MB 43.4 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
reviewsPro = riot_df['pros']      # pros column
reviewsCon = riot_df['cons']      # cons column

Now that we have our data, we need to parse things into sentences. This is a bit tricky--there's no unified format for user reviews. Some people use periods to separate sentences. Some use hyphens. Some use both hyphens and periods, with additional hyphens. When the data is scraped from the Internet, newlines are obliterated, so we can't use those as clues.

Instead, we'll assume that if we encounter a period or an exclamation mark, we're ending a sentence. We'll also assume that if we encounter a hyphen with a space after it, we're ending a sentence. 

In [ ]:
proSentences = []
conSentences = []

def parseSentencesFromReviewsColumn(reviewsColumn, reviewsList):
  # Iterate through the review at each row in the given column.
  for (columnName, columnData) in reviewsColumn.iteritems():
    currSent = ""
    prevChar = columnData[0]
    # columnData = one full positive review section. Loop thru char by char
    for character in columnData:
      # If we encounter a period, assume sentence, but only if the built string is
      # not spaces and not empty.
      if character == '.':
        if currSent.strip("-. ") and not currSent.strip(",. ").isspace():
          reviewsList.append(currSent.strip("-."))
        currSent = ""
      # If we encounter a "- ", assume sentence.
      elif character == ' ' and prevChar == '-':
        if currSent.strip("-. ") and not currSent.strip(",. ").isspace():
          reviewsList.append(currSent.strip(".-"))
      
        currSent = ""
    
      # Keep building the string and keep track of the prev char.
      currSent += character
      prevChar = character
      
    # Append whatever was left if it wasn't already caught
    if currSent.strip("-. ") and not currSent.strip(",. ").isspace():
      reviewsList.append(currSent.strip(".-"))

# Run on positive and negative reviews
parseSentencesFromReviewsColumn(reviewsPro, proSentences)
parseSentencesFromReviewsColumn(reviewsCon, conSentences)

# Sanity check
print(proSentences[:15])
print(conSentences[:15])

['Company which respects its employees and is pleasure to work at', 'Good company overall', ' Perks and the environment are good too', 'Very talented people work there', ' The product is awesome', ' The brand and reputation make you feel a rockstar', ' Salaries are good and benefits above market', 'Good location and a beautiful place', ' Amazing job culture ', ' Focus on players ', ' Challenge convention', ' Company Perks ', ' Talents ', ' Disruptive atmosphere ', ' Feedback culture']
['Cannot think of a con', 'It has changed its procedures and it´s got too many policies and rules to follow', ' Which doesn´t help much with the criativity of the employers', ' What used to take a few days to get settled it takes too long loosing the purpose of the project most of the times', "If you don't build a feel-good relationship with specific senior managers you will not thrive", " Nice people that don't do anything are better evaluated than hard working people rough around the edges", ' Meritocra

In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from pprint import pprint

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
lemmatizer = WordNetLemmatizer()

def tokenizeLemmetize(reviews):
  temp = []
  for sentence in reviews:
    tokens = word_tokenize(sentence)
    cleanedSentence = ""
    for token in tokens:
        lemmetized_word = lemmatizer.lemmatize(token)
        cleanedSentence += lemmetized_word + " "
    temp.append(cleanedSentence)
  return temp

In [ ]:
proSentencesCleaned = tokenizeLemmetize(proSentences)
conSentencesCleaned = tokenizeLemmetize(conSentences)

# Sanity check
print(proSentencesCleaned[:15])
print(conSentencesCleaned[:15])

['Company which respect it employee and is pleasure to work at ', 'Good company overall ', 'Perks and the environment are good too ', 'Very talented people work there ', 'The product is awesome ', 'The brand and reputation make you feel a rockstar ', 'Salaries are good and benefit above market ', 'Good location and a beautiful place ', 'Amazing job culture ', 'Focus on player ', 'Challenge convention ', 'Company Perks ', 'Talents ', 'Disruptive atmosphere ', 'Feedback culture ']
['Can not think of a con ', 'It ha changed it procedure and it´s got too many policy and rule to follow ', 'Which doesn´t help much with the criativity of the employer ', 'What used to take a few day to get settled it take too long loosing the purpose of the project most of the time ', "If you do n't build a feel-good relationship with specific senior manager you will not thrive ", "Nice people that do n't do anything are better evaluated than hard working people rough around the edge ", 'Meritocracy is a myth 

Now we're all set up to classify our sentences. We'll sort them into lists according to their valence and category (the six categories are given under "Zero-shot classification)--12 lists in total.

In [ ]:
# These are the possible categories of relevance we have defined.
# Diversity and inclusion = 1
# Culture and values = 2
# Work life balance = 3
# Senior management = 4
# Career opportunities = 5
# Compensation and benefits = 6
candidate_labels = ['diversity and inclusion', 'culture and values', 'work life balance', 'senior management', 'career opportunities', 'compensation and benefits']
pro1 = []
con1 = []
pro2 = []
con2 = []
pro3 = []
con3 = []
pro4 = []
con4 = []
pro5 = []
con5 = []
pro6 = []
con6 = []

pros = [pro1, pro2, pro3, pro4, pro5, pro6]
cons = [con1, con2, con3, con4, con5, con6]

# Let's be picky and assume that if the top value is lower than 0.4, the
# sentence is not relevant.

def sortReviewSentencesUsingZeroShot(sentenceList, labeledContainers):
  for sentence in sentenceList:
    cat = classifier(sentence, candidate_labels)
    if float(cat['scores'][0]) > 0.4:
      label = cat['labels'][0]
      if label == candidate_labels[0]:
        labeledContainers[0].append(sentence)
      elif label == candidate_labels[1]:
        labeledContainers[1].append(sentence)
      elif label == candidate_labels[2]:
        labeledContainers[2].append(sentence)
      elif label == candidate_labels[3]:
        labeledContainers[3].append(sentence)
      elif label == candidate_labels[4]:
        labeledContainers[4].append(sentence)
      elif label == candidate_labels[5]:
        labeledContainers[5].append(sentence)

In [ ]:
# Libraries needed to import/export files from/to drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Each list item is written on a separate line: lists within the list are 
# separated with the token "[LISTSEP]". For the filepath, you need to 
# input a directory that already exists in your drive. (e.g., 
# /content/drive/MyDrive/folderYouCreated/fileNameYouWant)

def writeListOfListsToFile(listThingy, filePath):
  with open(filePath, 'w') as writefile:
    for oneList in listThingy:
      for element in oneList:
        writefile.write(element)
        writefile.write('\n')
      writefile.write("[LISTSEP]\n")

In [ ]:
sortReviewSentencesUsingZeroShot(proSentencesCleaned, pros)
writeListOfListsToFile(pros, '/content/drive/MyDrive/compling_final/riotProsClassified.txt')

In [ ]:
sortReviewSentencesUsingZeroShot(conSentencesCleaned, cons)
writeListOfListsToFile(cons, '/content/drive/MyDrive/compling_final/riotConsClassified.txt')

Now we'll print some classifiers and store them for our confusion matrix.

In [ ]:
def printClassifiersForConfusion(sentenceList, howMany, filePath):
    # print(classifier(sentenceList[count], candidate_labels))
  with open(filePath, 'w') as writefile:
    for i in range(howMany):
      writefile.write(str(classifier(sentenceList[i], candidate_labels)))
      writefile.write("\n")

In [ ]:
printClassifiersForConfusion(proSentencesCleaned, 25, '/content/drive/MyDrive/compling_final/riotPosConfusion.txt')

In [ ]:
printClassifiersForConfusion(conSentencesCleaned, 25, '/content/drive/MyDrive/compling_final/riotNegConfusion.txt')

### Retroactive date handling (pt. 2)

Producing a "parallel" text file of dates that, line-by-line, matches each sentence from the classification file with its authorship date.

In [17]:
print(dates_dict_pros)

{'Company which respect it employee and is pleasure to work at ': '07/10/2018', 'Good company overall ': '06/27/2018', 'Perks and the environment are good too ': '06/27/2018', 'Very talented people work there ': '04/27/2018', 'The product is awesome ': '04/27/2018', 'The brand and reputation make you feel a rockstar ': '04/27/2018', 'Salaries are good and benefit above market ': '04/27/2018', 'Good location and a beautiful place ': '04/22/2018', 'Amazing job culture ': '04/23/2018', 'Focus on player ': '04/23/2018', 'Challenge convention ': '04/23/2018', 'Company Perks ': '04/10/2018', 'Talents ': '04/10/2018', 'Disruptive atmosphere ': '04/10/2018', 'Feedback culture ': '01/24/2018', 'Even already established since late 2013 , the Hong Kong Studio still breathes Start-Up ': '02/20/2018', 'It is small scale , we shifted away from being a China Publishing Office to become and build the first International Studio 2 year ago ': '02/20/2018', 'If you like to work in a Start-up environment 

In [18]:
print(dates_dict_cons)

{'Can not think of a con ': '07/10/2018', 'It ha changed it procedure and it´s got too many policy and rule to follow ': '06/27/2018', 'Which doesn´t help much with the criativity of the employer ': '06/27/2018', 'What used to take a few day to get settled it take too long loosing the purpose of the project most of the time ': '06/27/2018', "If you do n't build a feel-good relationship with specific senior manager you will not thrive ": '04/27/2018', "Nice people that do n't do anything are better evaluated than hard working people rough around the edge ": '04/27/2018', 'Meritocracy is a myth ': '04/27/2018', 'nothing so many important to say ': '04/22/2018', 'Nothing , is the dream company ': '04/23/2018', 'No diversity ': '04/10/2018', 'Lack of management for people who work during weekend ': '04/10/2018', 'Lack of work-life balance ': '07/31/2013', 'We are still partly dependent on the Central product team at the moment wrt the product we are working on in Hong Kong ': '02/20/2018',

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
category_delimiter = "[LISTSEP]\n"

def match_dates(filename_read, dates_dict):

  # Reading from file of classified review sentences
  f_read = open(filename_read, "r")
  classified_lines = f_read.readlines()

  dates_in_classification_order = []

  for line in classified_lines:

    if line != category_delimiter:
      # Remove trailing newline & leading whitespacing (single space)
      line_stripped = line.rstrip('\n').lstrip(' ')
      date_string = dates_dict[line_stripped]
      dates_in_classification_order.append(date_string)

    else:
      dates_in_classification_order.append(category_delimiter)

  f_read.close()
  
  return dates_in_classification_order

In [21]:
dates_pros = match_dates('/content/drive/MyDrive/new_sony/riotProsClassified_FIXED.txt', dates_dict_pros)

In [22]:
print(dates_pros)

['06/26/2014', '01/08/2018', '11/09/2016', '11/09/2016', '09/06/2015', '12/07/2015', '12/07/2015', '09/26/2016', '09/24/2015', '01/17/2015', '10/31/2013', '04/20/2012', '04/20/2012', '04/20/2012', '02/21/2018', '01/15/2018', '01/28/2016', '09/24/2015', '01/11/2018', '05/17/2018', '05/17/2018', '11/22/2016', '03/04/2018', '10/26/2012', '04/18/2014', '04/18/2014', '12/16/2014', '01/23/2018', '05/04/2012', '08/22/2014', '08/22/2014', '06/29/2015', '10/25/2015', '10/25/2015', '11/04/2015', '08/12/2012', '08/12/2012', '08/12/2012', '04/25/2015', '03/05/2015', '12/16/2014', '12/10/2014', '11/28/2014', '12/26/2013', '12/26/2013', '08/22/2014', '08/22/2014', '06/30/2014', '05/20/2014', '05/09/2014', '05/09/2014', '04/18/2014', '04/18/2014', '12/15/2013', '07/17/2013', '07/11/2013', '07/11/2013', '10/12/2009', '09/09/2012', '09/09/2012', '08/12/2012', '08/12/2012', '08/12/2012', '08/12/2014', '09/08/2016', '01/17/2015', '05/25/2021', '04/01/2021', '03/29/2021', '12/06/2019', '03/07/2019', '01/1

In [ ]:
dates_cons = match_dates('/content/drive/MyDrive/classifiedData/riotConsClassified.txt', dates_dict_cons)

In [ ]:
print(dates_cons)

In [23]:
# Input:
#   (1) filepath
#   (2) resulting list of classification-ordered dates from match_dates
# Output:
#   Does not return anything, but writes dates to specified file(path).
def write_dates_to_file(filename_write, dates_ordered):

  f_write = open(filename_write, "w")

  for date_string in dates_ordered:
    # If [LISTSEP], don't write in newline
    if date_string[0] == "[":
      f_write.write("%s" % date_string)
    else:
      f_write.write("%s\n" % date_string)
  
  f_write.close()

In [24]:
# Pros
write_dates_to_file('/content/drive/MyDrive/dates/riotPosDates.txt', dates_pros)

In [ ]:
# Cons
write_dates_to_file('/content/drive/MyDrive/dates/riotNegDates.txt', dates_cons)